In [1]:
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch

import time

import warnings
warnings.filterwarnings('ignore')

In [2]:
#-----------------GPU_play---------------
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
#------------------dataset----------------
train_data = CIFAR10('/export/space/okamoto-ka/jupyter/notebook/report6/kadai1',
                   train=True,download=True,transform=transforms.ToTensor())
test_data = CIFAR10('/export/space/okamoto-ka/jupyter/notebook/report6/kadai1',
                   train=False,download=True,transform=transforms.ToTensor())
#------------------cpu_loader----------------
cpu_train_loader = DataLoader(train_data,batch_size=50,shuffle=True)
cpu_test_loader = DataLoader(test_data,batch_size=50,shuffle=False)
#------------------gpu_loader----------------
gpu_train_loader = DataLoader(train_data,batch_size=100,shuffle=True)
gpu_test_loader = DataLoader(test_data,batch_size=100,shuffle=False)
#------------------gpu_loader----------------
gpu_Quad_train_loader = DataLoader(train_data,batch_size=200,shuffle=True)
gpu_Quad_test_loader = DataLoader(test_data,batch_size=200,shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
print(test_data)

Dataset CIFAR10
    Number of datapoints: 10000
    Split: test
    Root Location: /export/space/okamoto-ka/jupyter/notebook/report6/kadai1
    Transforms (if any): ToTensor()
    Target Transforms (if any): None


In [5]:
#--------------define model---------------------
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [6]:
def learning(net,gpu_use=False):
    #-------------------train-------------------------
    net.train()
    since = time.time() 
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(),lr=0.001,momentum=0.9)
    bestloss = 100
    
    if gpu_use: 
          mode = 'gpu'
          train_loader = gpu_train_loader
          test_loader = gpu_train_loader
          mini_batches = 80   
    else:
          mode = 'cpu'
          train_loader = cpu_train_loader
          test_loader =  cpu_test_loader
          mini_batches = 160
        
    print('strat traing mode:%s'%mode)
    
    for epoch in range(12):
        
     running_loss = 0.0   
     for i ,(inputs,labels) in enumerate(train_loader):
       #Variable
       if gpu_use: 
          inputs = inputs.to(device)
          labels = labels.to(device)
       #zero_grad
       optimizer.zero_grad()
       #Forward
       outputs = net(inputs)
       #loss 
       loss = criterion(outputs,labels)
       #backward
       loss.backward()
       #param update
       optimizer.step()
       running_loss += loss.item()

       if i %mini_batches == (mini_batches-1):
          if running_loss < bestloss:
            bestloss = running_loss
          
          running_loss = 0.0

       time_elapsed = time.time()-since
       
    print('Finished Training ')
    print('Bset loss : %.4f'%( bestloss/mini_batches))
    print('Training compete in %d m %.3f s \n' %(time_elapsed//60,time_elapsed % 60))
    
    #-------------------test-------------------------
    net.eval()
    correct = 0
    total = 0 
    with torch.no_grad():
     for (inputs,labels) in test_loader:
      if gpu_use: 
        inputs = inputs.to(device)
        labels = labels.to(device)
       
      outputs = net(inputs)
      _, predicted = torch.max(outputs.data,1)
      total +=labels.size(0)
      correct += (predicted == labels).sum()
    print('Accuracy %d / %d = %.3f' %(correct, total, float(correct.data[0]) / total))
    

In [60]:
#CPUNet
cpu_net = Net()
gpu_mode = False
learning(cpu_net,gpu_mode)


#------------gpu--------------

if torch.cuda.device_count() >1:
 #GPUNet   
 print("Let's use", torch.cuda.device_count(), "GPUs!")
 gpu_net = Net()
 gpu_mode =  True   
 gpu_net = nn.DataParallel(gpu_net)
 gpu_net = gpu_net.to(device)
 learning(gpu_net,gpu_mode)

  

strat traing mode:cpu
Finished Training 
Bset loss : 0.6250
Training compete in 2 m 37.811 s 

Accuracy 5127 / 10000 = 0.513
Let's use 2 GPUs!
strat traing mode:gpu
Finished Training 
Bset loss : 1.2500
Training compete in 1 m 6.407 s 

Accuracy 22888 / 50000 = 0.458


In [8]:
#------------gpu--------------

if torch.cuda.device_count() >1:
 #GPUNet   
 print("Let's Quad GPU use", torch.cuda.device_count(), "GPUs!")
 gpu_quad_net = Net()
 gpu_mode =  True   
 gpu_quad_net = nn.DataParallel(gpu_quad_net)
 gpu_net = gpu_quad_net.to(device)
 learning(gpu_quad_net,gpu_mode)


Let's Quad GPU use 4 GPUs!
strat traing mode:gpu
Finished Training 
Bset loss : 1.2500
Training compete in 1 m 6.601 s 

Accuracy 20637 / 50000 = 0.413
